In [328]:
import pandas as pd
import numpy as np
import math
from geopy.geocoders import Nominatim
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

### Map monde, sympa pour l'oral, remplacer 'equirectangular' par 'orthographic'

In [329]:
#Plotly with jupyter offline : 
init_notebook_mode(connected=True)

In [330]:
df = pd.read_csv('map.csv', index_col=0, names = ['nb_attack', 'target', 'attacker'], header = 0)

In [331]:
df.head()

,nb_attack,target,attacker
0,92157,united states,united states
1,26877,united arab emirates,china
2,13059,united states,china
3,6424,united states,ukraine
4,5202,united arab emirates,netherlands


In [332]:
df_country = pd.DataFrame(list(set(list(df['attacker'].unique()) + list(df['target'].unique()))), columns = ['country'] )

In [333]:
geolocator = Nominatim()
df_country['lattitude_longitude'] = df_country['country'].apply(lambda x : geolocator.geocode(x,  timeout=10)[1])

In [334]:
# Geolocator is wrong for united arabe emirates 
df_country['lattitude_longitude'] = df_country.apply(lambda x : (25.61552, 55.84951) if x['country'] == 'united arab emirates' else x['lattitude_longitude'], axis = 'columns')
df_country['lattitude_longitude'] = df_country.apply(lambda x : (39.904211, 116.407395) if x['country'] == 'china' else x['lattitude_longitude'], axis = 'columns')
df_country['lattitude_longitude'] = df_country.apply(lambda x : (37.566535, 126.9779692) if x['country'] == 'south korea' else x['lattitude_longitude'], axis = 'columns') 
df_country['lattitude_longitude'] = df_country.apply(lambda x : (51.3030, 0.0732) if x['country'] == 'united kingdom' else x['lattitude_longitude'], axis = 'columns') 
df_country['lattitude_longitude'] = df_country.apply(lambda x : (40.18658704808404, 44.78151767883003) if x['country'] == 'armenia' else x['lattitude_longitude'], axis = 'columns') 

In [335]:
df_country['latitude_attacker'] = df_country['lattitude_longitude'].apply(lambda x : x[0])
df_country['longitude_attacker'] = df_country['lattitude_longitude'].apply(lambda x : x[1])
df_country['attacker'] = df_country['country']
df_country = df_country.drop(['lattitude_longitude', 'country'], axis = 'columns')

In [336]:
df = pd.merge(df, df_country, how = 'left', on = ['attacker'])

In [337]:
df_country['latitude_target'] = df_country['latitude_attacker']
df_country['longitude_target'] = df_country['longitude_attacker']
df_country['target'] = df_country['attacker']
df_country = df_country.drop(['longitude_attacker', 'attacker', 'latitude_attacker'], axis = 'columns')

In [338]:
df = pd.merge(df, df_country, how = 'left', on = ['target'])

In [339]:
df.head()

,nb_attack,target,attacker,latitude_attacker,longitude_attacker,latitude_target,longitude_target
0,92157,united states,united states,39.783730,-100.445882,39.78373,-100.445882
1,26877,united arab emirates,china,39.904211,116.407395,25.61552,55.849510
2,13059,united states,china,39.904211,116.407395,39.78373,-100.445882
3,6424,united states,ukraine,49.487197,31.271832,39.78373,-100.445882
4,5202,united arab emirates,netherlands,52.237989,5.534607,25.61552,55.849510


In [340]:
df['lat'] = df.apply(lambda x :[x['latitude_attacker'], x['latitude_target']], axis ='columns')
df['lon'] =  df.apply(lambda x :[x['longitude_attacker'], x['longitude_target']], axis ='columns')

In [341]:
df['line'] = ''
#df['line'] = df['line'].apply(lambda x : {'color' : 'red', 'width' : 1}) 
df['line'] = df.apply(lambda x : {'color' : 'red', 'width' : (x['nb_attack'] / sum(df['nb_attack']))*80}, axis='columns')

In [342]:
df['opacity'] = 0
df['opacity'] = df['opacity'].apply(lambda x : 0.48577680525164113)

In [343]:
df['type'] = ''
df['type'] = df['type'].apply(lambda x : 'scattergeo')

In [344]:
df['mode'] = ''
df['mode'] = df['mode'].apply(lambda x : 'lines')

In [345]:
df['text'] = df.apply(lambda x : '{} - {} : {} times'.format(x['attacker'],x['target'], x['nb_attack']), axis='columns')

In [417]:
df[df['attacker'] == 'france']

,nb_attack,target,attacker,latitude_attacker,longitude_attacker,latitude_target,longitude_target,lat,lon,line,opacity,type,mode,text,showlegend
22,1093,france,france,46.603354,1.888334,46.603354,1.888334,"[46.603354, 46.603354]","[1.8883335, 1.8883335]","{'width': 0.40423466321483054, 'color': 'red'}",0.485777,scattergeo,lines,france - france : 1093 times,False
32,812,united arab emirates,france,46.603354,1.888334,25.615520,55.849510,"[46.603354, 25.61552]","[1.8883335, 55.84951]","{'width': 0.30030974064999305, 'color': 'red'}",0.485777,scattergeo,lines,france - united arab emirates : 812 times,False
80,267,united states,france,46.603354,1.888334,39.783730,-100.445882,"[46.603354, 39.7837304]","[1.8883335, -100.4458825]","{'width': 0.0987471684156997, 'color': 'red'}",0.485777,scattergeo,lines,france - united states : 267 times,False
113,142,italy,france,46.603354,1.888334,42.638426,12.674297,"[46.603354, 42.6384261]","[1.8883335, 12.674297]","{'width': 0.05251722065554066, 'color': 'red'}",0.485777,scattergeo,lines,france - italy : 142 times,False


In [347]:
df['showlegend'] = False
data_norse = df[['lat', 'line', 'lon', 'mode', 'opacity','type', 'showlegend']].to_dict(orient='records')
df_intra_pays = df[df['attacker'] == df['target']]

labels = [ dict(
        type = 'scattergeo',
        hoverinfo = 'text',
        lon = df['longitude_attacker'],
        lat = df['latitude_attacker'],
        text = df['attacker'],
        mode = 'markers',
        marker = dict( 
            size=2, 
            color='rgb(255, 0, 0)',
            line = dict(
                width=1,
                color='rgba(68, 68, 68, 0)'
            )   
        ),
        legendgroup = 'group1', # this can be any string, not just "group"
        name = 'Flux inter pays',
        showlegend = True
    
)]

cases = []
cases.append(go.Scattergeo(
    hoverinfo = 'text',
    lon = df_intra_pays['longitude_attacker'], 
    lat = df_intra_pays['latitude_attacker'],
    text = df_intra_pays['attacker'],
    marker = dict(
        size = df_intra_pays['nb_attack'].apply(lambda x :  x / sum(df['nb_attack']) * 80),
        color = 'rgb(33,113,181)',
        line = dict(width = 0)
    ),
    legendgroup = 'group2', # this can be any string, not just "group"
    name = 'Flux intra pays',
    mode = 'markers',
    showlegend = True
    
))

 
layout = dict(
    title = 'Flux des cyberattaques mondiale en mars 2018 <br> source : <a href="http://www.norse-corp.com/about-us/who-we-are/">\
Norse </a>',
        
        geo = dict(
            scope='world',
            projection= dict( type='orthographic'),
            showland = True,
            landcolor = 'rgb(243, 243, 243)',
            countrycolor = 'rgb(204, 204, 204)',
            showcountries = True
        )
    )
    
fig = dict(data = data_norse + cases + labels, layout=layout )
iplot(fig, filename='d3-world-map')

In [348]:
# 1ère cartes : Représentation des cyber attaques 
# 2 ème carte : cibles des cyber-attaques 
# 3 ème carte : origine des attaques 

In [349]:
df_code_country = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')[['CODE', 'COUNTRY']]
df_code_country['country'] = df_code_country['COUNTRY'].apply(lambda x : x.lower())
df_code_country = df_code_country.drop(['COUNTRY'], axis = 'columns')

In [350]:
sum_target = df.groupby(['target'])['nb_attack'].sum()
df_target = df.set_index(['target'])[['latitude_target', 'longitude_target']]
df_target['latitude'] = df_target['latitude_target']
df_target['longitude'] = df_target['longitude_target']
df_target['Nb_attack'] = sum_target
df_target = df_target.reset_index()
df_target = df_target.drop_duplicates('target')
df_target['country'] = df_target['target']
df_target = df_target.drop(['target', 'latitude_target','longitude_target'], axis = 'columns')

In [351]:
df_target = pd.merge(df_target, df_code_country, how = 'left', on = ['country'])

In [352]:
df_target.head()

,latitude,longitude,Nb_attack,country,CODE
0,39.783730,-100.445882,137702,united states,USA
1,25.615520,55.849510,45635,united arab emirates,ARE
2,40.002803,-4.003104,7740,spain,ESP
3,1.290475,103.852036,2702,singapore,SGP
4,64.573154,11.528036,1828,norway,NOR


In [353]:
sum_attacker = df.groupby(['attacker'])['nb_attack'].sum()
df_attacker = df.set_index(['attacker'])[['latitude_attacker', 'longitude_attacker']]
df_attacker['sum_attacker'] = sum_attacker
df_attacker = df_attacker.reset_index()
df_attacker = df_attacker.drop_duplicates('attacker')
df_attacker['country'] = df_attacker['attacker']
df_attacker = df_attacker.drop(['attacker'], axis = 'columns')

In [354]:
df_attacker = pd.merge(df_attacker, df_code_country, how = 'left', on = ['country'])

In [355]:
df_attacker.head()

,latitude_attacker,longitude_attacker,sum_attacker,country,CODE
0,39.783730,-100.445882,97988,united states,USA
1,39.904211,116.407395,52725,china,CHN
2,49.487197,31.271832,6680,ukraine,UKR
3,52.237989,5.534607,7854,netherlands,NLD
4,22.351115,78.667743,3183,india,IND


In [358]:
colorscale=[[0, "#f7fbff"], [0.25, "#FE96A0"], [0.45, "#FF6F7D"], [0.65, "#FF6F7D"], [0.85, "#DB1702"], [1,"#D90115"]]


data = [ dict(
        type='choropleth',
        colorscale = colorscale,
        autocolorscale = False,
        locations = df_target['CODE'],
        z = np.log(df_target['Nb_attack']).astype(float),
        text = df_target['country'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 0
            ) ),
        colorbar = dict(
            title = "Cyberattaques <br>échelle logarithmique")
        ) ]

 
layout = dict(
    title = 'Cibles des cyberattaques -  mars 2018 <br> source : <a href="http://www.norse-corp.com/about-us/who-we-are/">\
Norse </a>',
        
        geo = dict(
            scope='world',
            projection= dict( type='equirectangular'),
            showland = True,
            landcolor = 'rgb(243, 243, 243)',
            countrycolor = 'rgb(204, 204, 204)',
            showcountries = True
        )
    )
    
fig = dict( data=data, layout=layout )
iplot( fig, filename='d3-cloropleth-map' )

In [416]:
colorscale=[[0, "#f7fbff"], [0.004, "#FE96A0"], [0.005, "#FF6F7D"], [0.03, "#FF6F7D"], [0.20, "#DB1702"], [1,"#D90115"]]



data = [ dict(
        type='choropleth',
        colorscale = colorscale,
        autocolorscale = False,
        locations = df_target['CODE'],
        z = df_target['Nb_attack'].astype(float),
        text = df_target['country'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 0
            ) ),
        colorbar = dict(
            title = "Cyberattaques")
        ) ]

layout = dict(
    title = 'Cibles des cyberattaques -  mars 2018 <br> source : <a href="http://www.norse-corp.com/about-us/who-we-are/">\
Norse </a>',
        
        geo = dict(
            scope='world',
            projection= dict( type='equirectangular'),
            showland = True,
            landcolor = 'rgb(243, 243, 243)',
            countrycolor = 'rgb(204, 204, 204)',
            showcountries = True
        )
    )
    
fig = dict( data=data, layout=layout )
iplot( fig, filename='d3-cloropleth-map' )

In [415]:
colorscale=[[1, 'rgb(165,0,38)'], [0.8888, 'rgb(215,48,39)'], [0.777, 'rgb(244,109,67)'], [0.666, 'rgb(253,174,97)'], [0.555, 'rgb(254,224,144)'], [0.444, 'rgb(224,243,248)'], [0.333, 'rgb(171,217,233)'], [0.222, 'rgb(116,173,209)'], [0.111, 'rgb(69,117,180)'], [0, 'rgb(49,54,149)']]
    
data = [ dict(
        type='choropleth',
        colorscale = colorscale,
        autocolorscale = False,
        locations = df_attacker['CODE'],
        z = np.log(df_attacker['sum_attacker']).astype(float),
        text = df_attacker['country'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 0
            ) ),
        colorbar = dict(
            title = "Cyberattaques <br>échelle logarithmique")
        ) ]

 
layout = dict(
    title = 'Origine des cyberattaques - mars 2018 <br> source : <a href="http://www.norse-corp.com/about-us/who-we-are/">\
Norse </a>',
        
        geo = dict(
            scope='world',
            projection= dict( type='equirectangular'),
            showland = True,
            landcolor = 'rgb(243, 243, 243)',
            countrycolor = 'rgb(204, 204, 204)',
            showcountries = True
        )
    )
    
fig = dict( data=data, layout=layout )
iplot( fig, filename='d3-cloropleth-map' )

In [360]:
df_zone_h = pd.read_csv('./../CSV_for_analyse/NbrAttack_Pays_ZoneH.csv', encoding = 'latin-1',index_col=0)

In [361]:
df_zone_h.head()

,Pays,Nb_attack
0,united states,34734
1,brazil,8586
2,united kingdom,6128
3,germany,4436
4,china,2770


In [362]:
df_country = pd.DataFrame(list(set(df_zone_h['Pays'])), columns = ['Pays'])

In [363]:
geolocator = Nominatim()
df_country['lattitude_longitude'] = df_country['Pays'].apply(lambda x : geolocator.geocode(x,  timeout=10)[1])

In [364]:
# Geolocator is wrong for united arabe emirates 
df_country['lattitude_longitude'] = df_country.apply(lambda x : (25.61552, 55.84951) if x['Pays'] == 'united arab emirates' else x['lattitude_longitude'], axis = 'columns')
df_country['lattitude_longitude'] = df_country.apply(lambda x : (39.904211, 116.407395) if x['Pays'] == 'china' else x['lattitude_longitude'], axis = 'columns')
df_country['lattitude_longitude'] = df_country.apply(lambda x : (37.566535, 126.9779692) if x['Pays'] == 'south korea' else x['lattitude_longitude'], axis = 'columns') 
df_country['lattitude_longitude'] = df_country.apply(lambda x : (51.3030, 0.0732) if x['Pays'] == 'united kingdom' else x['lattitude_longitude'], axis = 'columns') 
df_country['lattitude_longitude'] = df_country.apply(lambda x : (40.18658704808404, 44.78151767883003) if x['Pays'] == 'armenia' else x['lattitude_longitude'], axis = 'columns') 

In [365]:
df_country['latitude'] = df_country['lattitude_longitude'].apply(lambda x : x[0])
df_country['longitude'] = df_country['lattitude_longitude'].apply(lambda x : x[1])

In [366]:
df_country.head()

,Pays,lattitude_longitude,latitude,longitude
0,mexico,"(19.4326009, -99.1333416)",19.432601,-99.133342
1,cyprus,"(34.9823018, 33.1451285)",34.982302,33.145128
2,israel,"(30.8760272, 35.0015196)",30.876027,35.001520
3,NaN,"(45.1674908, 5.4175633)",45.167491,5.417563
4,pakistan,"(30.3308401, 71.247499)",30.330840,71.247499


In [367]:
df_zone_h = pd.merge(df_zone_h, df_country, how = 'left', on= ['Pays']).dropna()

In [368]:
df_zone_h['country'] = df_zone_h['Pays']
df_zone_h = df_zone_h.drop(['Pays'], axis = 'columns')

In [369]:
df_zone_h = pd.merge(df_zone_h, df_code_country, how = 'left', on = ['country'])[['Nb_attack', 'latitude', 'longitude', 'country', 'CODE']]

In [418]:
colorscale=[[0, "#f7fbff"], [0.25, "#FE96A0"], [0.45, "#FF6F7D"], [0.65, "#FF6F7D"], [0.85, "#DB1702"], [1,"#D90115"]]

data = [ dict(
        type='choropleth',
        colorscale = colorscale,
        autocolorscale = False,
        locations = df_zone_h['CODE'],
        z = np.log(df_zone_h['Nb_attack']).astype(float),
        text = df_zone_h['country'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 0
            ) ),
        colorbar = dict(
            title = "Cyberattaques <br>échelle logarithmique")
        ) ]

 
layout = dict(
    title = 'Cibles des cyberattaques - depuis 1998 <br> source : <a href="http://zone-h.org/?zh=1">\
Zone H </a>' ,
        
        geo = dict(
            scope='world',
            projection= dict( type='equirectangular'),
            showland = True,
            landcolor = 'rgb(243, 243, 243)',
            countrycolor = 'rgb(204, 204, 204)',
            showcountries = True
        )
    )
    
fig = dict( data=data, layout=layout )
iplot( fig, filename='d3-cloropleth-map' )

In [419]:
df_zone_h

,Nb_attack,latitude,longitude,country,CODE
0,34734,39.783730,-100.445882,united states,USA
1,8586,-10.333333,-53.200000,brazil,BRA
2,6128,51.303000,0.073200,united kingdom,GBR
3,4436,51.083420,10.423447,germany,DEU
4,2770,39.904211,116.407395,china,CHN
5,2702,37.566535,126.977969,south korea,NaN
6,2524,19.432601,-99.133342,mexico,MEX
7,2518,23.598298,120.835363,taiwan,TWN
8,2220,46.603354,1.888334,france,FRA
9,1768,52.097718,19.025816,poland,POL


In [371]:
df_target_norse_zone_h = pd.concat([df_target, df_zone_h])

In [372]:
sum_attack = df_target_norse_zone_h.groupby(['country'])['Nb_attack'].sum()
df_target_norse_zone_h = df_target_norse_zone_h.drop(['Nb_attack'], axis = 'columns')

In [373]:
df_target_norse_zone_h = pd.merge(df_target_norse_zone_h.drop_duplicates(['country']), sum_attack.reset_index(), how = 'left', on = ['country'])

In [432]:
colorscale=[[0, "#f7fbff"], [0.25, "#FE96A0"], [0.45, "#FF6F7D"], [0.65, "#FF6F7D"], [0.85, "#DB1702"], [1,"#D90115"]]

data = [ dict(
        type='choropleth',
        colorscale = colorscale,
        autocolorscale = False,
        locations = df_target_norse_zone_h['CODE'],
        z = np.log(df_target_norse_zone_h['Nb_attack']).astype(float),
        text = df_target_norse_zone_h['country'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 0
            ) ),
        colorbar = dict(
            title = "Cyberattaques <br>échelle logarithmique")
        ) ]

 
layout = dict(
    title = 'Cibles des cyberattaques <br> fusion des sources : <a href="http://zone-h.org/?zh=1">\
Zone H </a> et <a href="http://www.norse-corp.com/about-us/who-we-are/">\
Norse',
        
        geo = dict(
            scope='world',
            projection= dict( type=''),
            showland = True,
            landcolor = 'rgb(243, 243, 243)',
            countrycolor = 'rgb(204, 204, 204)',
            showcountries = True
        ),
    )



fig = dict( data=data, layout=layout )
iplot(fig, filename='d3-cloropleth-map' )